In [ ]:
import spike_data_augmentation
import spike_data_augmentation.transforms as transforms
from utils.helper import get_all_file_paths
from joblib import Parallel, delayed
from multiprocessing import Process, Manager
from math import sqrt
import ipdb
import os
import numpy as np
import time

In [ ]:
surface_dimensions = [9,9]
time_constant = 5e3
transform = transforms.ToTimesurface(surface_dimensions=surface_dimensions, tau=time_constant, decay='lin', merge_polarities=True)

testset = spike_data_augmentation.datasets.NMNIST(save_to='./data', train=True, download=True, transform=transform)

In [ ]:
import multiprocessing
from time import gmtime, strftime
print(strftime("Started on %a, %d %b %Y %H:%M:%S", gmtime()))
start_time = time.time()


def write_recording(data):
    recording, label, sensor_size, ordering, index, surface_dimensions, folder = data
    surf,_ = transform(recording, sensor_size, ordering)
    surf = surf.reshape([-1,] + surface_dimensions)
    file_name = './' + folder + str(index) + '_' + str(label) + '.npy'
    np.save(file_name, surf)
    
folder = 'surf_{0}x{1}_tau_{2}/'.format(*surface_dimensions, int(time_constant))

if __name__ == '__main__':
    with multiprocessing.Pool(100) as pool:
        if not os.path.exists(folder):
            os.makedirs(folder)
        overall = len(testset)
        combination = zip(testset.data,
                          testset.targets, 
                          [testset.sensor_size] * overall, 
                          [testset.ordering] * overall, 
                          np.arange(0,overall),
                          [surface_dimensions] * overall,
                          [folder] * overall)
        all_surfaces = pool.map(write_recording, combination)
        print(len(all_surfaces))
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
all_paths = get_all_file_paths(folder)

In [ ]:
with open(folder + '_all_paths.txt', 'w') as f:
    for item in all_paths:
        f.write("%s\n" % item)